In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/drive', force_remount=True)
import os
os.chdir('/drive/My Drive')
os.listdir()
path=('drive/My drive/traindata-openai')
#!ls

Mounted at /drive


In [ ]:
import re
df_yt = pd.read_csv('traindata-openai.csv')
df_yt = df_yt.drop_duplicates(subset = ['title'])
df_yt = df_yt[['title', 'description','label']]
df_yt.columns = ['title', 'description','label']
df_yt['cleaned_title'] = df_yt['title'].apply(lambda x: x.lower())
df_yt['cleaned_title'] = df_yt['cleaned_title'].apply(lambda x:re.sub('[^A-Za-z0-9]+', ' ', x))

In [ ]:
df_yt

,title,description,label,cleaned_title
0,58-STUDIO NINE KNITTED CARDIGA,STUDIO COLLECTION. Cardigan in a soft knit con...,Cardigan,58 studio nine knitted cardiga
1,Abingdon cardigan,"Cardigan in a soft rib knit with a round neck,...",Cardigan,abingdon cardigan
3,Abingdon Cardigan,Long-sleeved cardigan in a soft rib knit with ...,Cardigan,abingdon cardigan
4,Aja fancy hood,"Long-sleeved cardigan knitted in soft, garter-...",Cardigan,aja fancy hood
5,Aja hood,"Long-sleeved cardigan in soft, fine-knit cotto...",Cardigan,aja hood
...,...,...,...,...
1936,Zilla Printed Scarf TR,Scarf in a viscose weave with a floral print a...,Scarf,zilla printed scarf tr
1937,Zindra Recyled Scarf,Scarf in woven fabric made from recycled polye...,Scarf,zindra recyled scarf
1938,Zindra scarf conscious,Scarf in an airy weave made from recycled poly...,Scarf,zindra scarf conscious
1939,Zitta Scarf,Scarf in an airy weave with fringes along the ...,Scarf,zitta scarf


In [ ]:
!pip install tensorflow
!pip install tensorflow-hub
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 14.6 MB/s 
     |████████████████████████████████| 588.3 MB 17 kB/s 
     |████████████████████████████████| 439 kB 80.0 MB/s 
     |████████████████████████████████| 1.7 MB 68.9 MB/s 
     |████████████████████████████████| 6.0 MB 73.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
   

In [ ]:
import tensorflow as tf
import tensorflow_text as text  # Registers the ops.
import tensorflow_hub as hub

In [ ]:
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/albert_en_preprocess/3'
preprocessor = hub.KerasLayer(tfhub_handle_preprocess)
#preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1", trainable=True)

def get_bert_embeddings(text, preprocessor, encoder):

  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
  encoder_inputs = preprocessor(text_input)
  outputs = encoder(encoder_inputs)
  embedding_model = tf.keras.Model(text_input, outputs['pooled_output'])
  sentences = tf.constant([text])
  return embedding_model(sentences)

df_yt['encodings'] = df_yt['cleaned_title'].apply(lambda x: get_bert_embeddings(x, preprocessor, encoder))

In [ ]:
import re
import sklearn.metrics as metrics
def preprocess_text():
  text = input()
  text = text.lower()
  text = re.sub('[^A-Za-z0-9]+', ' ', text)
  return text
  
query_text = preprocess_text()
query_encoding = get_bert_embeddings(query_text, preprocessor, encoder)
df_yt['similarity_score'] = df_yt['encodings'].apply(lambda x: metrics.pairwise.cosine_similarity(x, query_encoding)[0][0])
df_results = df_yt.sort_values(by=['similarity_score'], ascending=False)
df_results

Balla silk scarf	


,title,description,label,cleaned_title,encodings,similarity_score
1547,Balla silk scarf,ANNA GLOVER x H&M. Scarf in patterned mulberry...,Scarf,balla silk scarf,"((tf.Tensor(-0.9983272, shape=(), dtype=float3...",1.000000
1798,PQ Balla silk scarf,Scarf in patterned silk. Size 80x80 cm.,Scarf,pq balla silk scarf,"((tf.Tensor(-0.99906117, shape=(), dtype=float...",0.996720
1799,PQ Ballic silk scarf,"Scarf in soft, gently draping, patterned silk.",Scarf,pq ballic silk scarf,"((tf.Tensor(-0.9991553, shape=(), dtype=float3...",0.996110
1818,PQ VICKY SILK SCARF,Double-layered scarf in patterned silk. Size 9...,Scarf,pq vicky silk scarf,"((tf.Tensor(-0.99979895, shape=(), dtype=float...",0.950388
69,Barcelona,"Cardigan in soft, textured fine-knit cotton wi...",Cardigan,barcelona,"((tf.Tensor(-0.9996436, shape=(), dtype=float3...",0.928164
...,...,...,...,...,...,...
1044,Hemingway cable hood,Jumper in a soft knit containing some wool wit...,Hoodie,hemingway cable hood,"((tf.Tensor(-0.99957764, shape=(), dtype=float...",0.530403
773,Austin Tech Hood,Sports jacket in jersey with a zip down the fr...,Hoodie,austin tech hood,"((tf.Tensor(-0.99969506, shape=(), dtype=float...",0.523639
1270,Oliver Tech Hoodie,Track jacket in heavy jersey with a zip down t...,Hoodie,oliver tech hoodie,"((tf.Tensor(-0.9998034, shape=(), dtype=float3...",0.507640
1268,Oliver tech hood,"Sports jacket in sturdy, double-layer jersey w...",Hoodie,oliver tech hood,"((tf.Tensor(-0.9996943, shape=(), dtype=float3...",0.480065


In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(true_labels, predictions)) # Value between 0 and 1

print("Accuracy Percentage {} %:".format(100*accuracy_score(true_labels, predictions))) # Value between 0 and 100
``
